In [1]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data=pd.read_csv("../artifacts/data/sp500_all.csv",skipfooter=569030)
data.head()

C:\Users\manth\AppData\Local\Temp\ipykernel_9984\3366406071.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  data=pd.read_csv("../artifacts/data/sp500_all.csv",skipfooter=569030)


,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50010 entries, 0 to 50009
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    50010 non-null  object 
 1   open    50010 non-null  float64
 2   high    50010 non-null  float64
 3   low     50010 non-null  float64
 4   close   50010 non-null  float64
 5   volume  50010 non-null  int64  
 6   Name    50010 non-null  object 
dtypes: float64(4), int64(1), object(2)
memory usage: 2.7+ MB


In [4]:
data.describe()


,open,high,low,close,volume
count,50010.000000,50010.000000,50010.000000,50010.000000,5.001000e+04
mean,95.044450,95.959338,94.097704,95.062318,5.000432e+06
std,104.566486,105.494946,103.517820,104.557724,1.221146e+07
min,1.620000,1.690000,1.610000,1.620000,7.802900e+04
25%,48.290000,48.750000,47.840000,48.290000,1.055344e+06
50%,67.285000,67.860000,66.730000,67.340000,1.982146e+06
75%,106.500000,107.675000,105.400000,106.547500,3.862249e+06
max,1477.390000,1498.000000,1450.040000,1450.890000,2.683365e+08


In [5]:
data.isna().sum()

date      0
open      0
high      0
low       0
close     0
volume    0
Name      0
dtype: int64

In [6]:
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(strategy='mean')
data['open']=imputer.fit_transform(data[['open']])
data['high']=imputer.fit_transform(data[['high']])
data['low']=imputer.fit_transform(data[['low']])

data.isna().sum()

date      0
open      0
high      0
low       0
close     0
volume    0
Name      0
dtype: int64

In [7]:
data.duplicated().sum()

np.int64(0)

In [8]:
from sklearn.preprocessing import  LabelEncoder
encoder=LabelEncoder()
data['Name']=encoder.fit_transform(data['Name'])


In [9]:
data['date'] = pd.to_datetime(data['date'])

# Day, month, year
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day

# Day of the week (0 = Monday, 6 = Sunday)
data['day_of_week'] = data['date'].dt.dayofweek

# Week of the year
data['week_of_year'] = data['date'].dt.isocalendar().week

# Quarter of the year
data['quarter'] = data['date'].dt.quarter

# Is month start/end
data['is_month_start'] = data['date'].dt.is_month_start.astype(int)
data['is_month_end'] = data['date'].dt.is_month_end.astype(int)

# Is quarter start/end
data['is_quarter_start'] = data['date'].dt.is_quarter_start.astype(int)
data['is_quarter_end'] = data['date'].dt.is_quarter_end.astype(int)



In [10]:
cols = ['open', 'high', 'low', 'close']
windows = [5, 10, 20]

for col in cols:
    for w in windows:
        data[f'{col}_EMA_{w}'] = data[col].ewm(span=w, adjust=False).mean()


In [11]:
data.head()

,date,open,high,low,close,volume,Name,year,month,day,...,open_EMA_20,high_EMA_5,high_EMA_10,high_EMA_20,low_EMA_5,low_EMA_10,low_EMA_20,close_EMA_5,close_EMA_10,close_EMA_20
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,0,2013,2,8,...,15.070000,15.120000,15.120000,15.120000,14.630000,14.630000,14.630000,14.750000,14.750000,14.750000
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,0,2013,2,11,...,15.052857,15.083333,15.100000,15.109524,14.506667,14.562727,14.594762,14.653333,14.697273,14.722381
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,0,2013,2,12,...,14.995442,14.892222,14.992727,15.052426,14.371111,14.478595,14.547642,14.525556,14.619587,14.679297
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,0,2013,2,13,...,14.929210,14.908148,14.983140,15.041719,14.330741,14.437032,14.519295,14.570370,14.626935,14.677459
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,0,2013,2,14,...,14.930237,14.925432,14.978933,15.033936,13.940494,14.204845,14.389838,14.376914,14.511128,14.611987


In [12]:
X=data.drop(columns=['date','close']).values
Y=data['close'].values

In [13]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, shuffle=False)

In [14]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [15]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, RandomizedSearchCV

# Regression models
from sklearn.linear_model import LinearRegression ,Ridge , Lasso, ElasticNet    
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor

# XGBoost, LightGBM, CatBoost
import xgboost as xgb
from xgboost import XGBRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# Metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
models={
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(),
    "KNeighborsRegressor": KNeighborsRegressor(),
    "DecisionTreeRegressor": DecisionTreeRegressor(),
    "RandomForestRegressor": RandomForestRegressor(),
    "GradientBoostingRegressor": GradientBoostingRegressor(),
    "AdaBoostRegressor": AdaBoostRegressor(),
    "BaggingRegressor": BaggingRegressor(),
    "XGBRegressor": XGBRegressor(objective='reg:squarederror'),
    "LGBMRegressor": LGBMRegressor(),
    "CatBoostRegressor": CatBoostRegressor(verbose=0),
    "SVR": SVR(),
}

# ===============================
# Comprehensive Hyperparameter Dictionary
# ===============================

param_grids_comprehensive = {

    # ----- Linear Models -----
    'LinearRegression': {},

    # ----- Tree / Ensemble Models -----
    'DecisionTreeRegressor': {
        'max_depth': [5, 10, 20, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 5],
        'max_features': [None, 'sqrt', 'log2'],
        'criterion': ['squared_error', 'absolute_error', 'poisson'],
        'splitter': ['best', 'random']
    },
    'Ridge': {
        'alpha': [0.1, 1.0, 10.0, 100.0],
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
        'fit_intercept': [True, False] 
    },
    'Lasso': {
        'alpha': [0.01, 0.1, 1.0, 10.0],
        'fit_intercept': [True, False]
    },
    'ElasticNet': {
        'alpha': [0.01, 0.1, 1.0, 10.0],
        'l1_ratio': [0.1, 0.5, 0.9],
        'fit_intercept': [True, False]
    },
    'RandomForestRegressor': {
        'n_estimators': [100, 200, 500],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['auto', 'sqrt', 'log2'],
        'bootstrap': [True, False],
        'criterion': ['squared_error', 'absolute_error']
    },
    'GradientBoostingRegressor': {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'subsample': [0.7, 1.0],
        'max_features': [None, 'sqrt', 'log2']
    },
    'AdaBoostRegressor': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.05, 0.1, 0.5, 1],
        'loss': ['linear', 'square', 'exponential']
    },
    'BaggingRegressor': {
        'n_estimators': [10, 50, 100],
        'max_samples': [0.5, 0.7, 1.0],
        'max_features': [0.5, 0.7, 1.0],
        'bootstrap': [True, False],
        'bootstrap_features': [True, False]
    },

    # ----- KNN / SVM -----
    'KNeighborsRegressor': {
        'n_neighbors': [3, 5, 7, 10],
        'weights': ['uniform', 'distance'],
        'p': [1, 2],
        'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
        'leaf_size': [20, 30, 40]
    },
   
    'SVR': {
        'kernel': ['linear','rbf', 'sigmoid'],
        'C': [0.1, 1, 10, 100],
    },
      

    # ----- Boosting Libraries -----
    'XGBRegressor': {
        'n_estimators': [100, 200, 500],
        'max_depth': [3, 5, 7, 10],
        'learning_rate': [0.01, 0.05, 0.1],
        'subsample': [0.7, 1],
        'colsample_bytree': [0.7, 1],
        'gamma': [0, 0.1, 0.3],
        'reg_alpha': [0, 0.01, 0.1],
        'reg_lambda': [1, 1.5, 2]
    },
    'LGBMRegressor': {
        'n_estimators': [100, 200, 500],
        'max_depth': [3, 5, 7, -1],
        'learning_rate': [0.01, 0.05, 0.1],
        'num_leaves': [31, 50, 100],
        'min_child_samples': [20, 30, 50],
        'subsample': [0.7, 1.0],
        'colsample_bytree': [0.7, 1.0],
        'reg_alpha': [0, 0.01, 0.1],
        'reg_lambda': [0, 0.1, 0.5]
    },
    'CatBoostRegressor': {
        'iterations': [500, 1000, 1500],
        'depth': [4, 6, 8, 10],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'l2_leaf_reg': [1, 3, 5, 7],
        'bagging_temperature': [0, 1, 2],
        'rsm': [0.7, 0.8, 1.0],
        'border_count': [32, 50, 100],
        'random_strength': [1, 2, 5],
        'verbose': [0]
    }
}



In [30]:
def model_training(models,param_grids,x_train,y_train,x_test,y_test):
    results = {}
    for name, model in models.items():
        print(f"Training {name}...")
        param_grid = param_grids.get(name, {})
        if param_grid:
            grid_search = RandomizedSearchCV(model, param_grid, cv=2, scoring='neg_mean_squared_error', n_jobs=-1)
            grid_search.fit(x_train, y_train)
            best_model = grid_search.best_estimator_
            best_params = grid_search.best_params_
            print(f"Best parameters for {name}: {best_params}")
        else:
            model.fit(x_train, y_train)
            best_model = model
            best_params = "Default parameters used"
        
        y_pred = best_model.predict(x_test)
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results[name] = {
            'model': best_model,
            'best_params': best_params,
            'mse': mse,
            'mae': mae,
            'r2': r2
        }
        
        print(f"{name} - MSE: {mse}, MAE: {mae}, R2: {r2}\n")
    
    return results

In [ ]:
model_training(models,param_grids_comprehensive,x_train,y_train,x_test,y_test)

Training LinearRegression...
LinearRegression - MSE: 0.5378904608556249, MAE: 0.3511777537602728, R2: 0.999985085108546

Training Ridge...
Best parameters for Ridge: {'solver': 'svd', 'fit_intercept': True, 'alpha': 0.1}
Ridge - MSE: 0.967285617447072, MAE: 0.4708886603205999, R2: 0.9999731786282913

Training Lasso...


d:\Projects\Portfolio Project\venv\lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
d:\Projects\Portfolio Project\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.597e+04, tolerance: 1.217e+04
  model = cd_fast.enet_coordinate_descent(


Best parameters for Lasso: {'fit_intercept': True, 'alpha': 0.01}
Lasso - MSE: 7.984974819640896, MAE: 1.3706224043420556, R2: 0.9997785886879121

Training ElasticNet...


d:\Projects\Portfolio Project\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.156e+04, tolerance: 1.217e+04
  model = cd_fast.enet_coordinate_descent(


Best parameters for ElasticNet: {'l1_ratio': 0.5, 'fit_intercept': True, 'alpha': 0.01}
ElasticNet - MSE: 27.97193651245541, MAE: 1.4259191588587021, R2: 0.9992243803763005

Training SVR...
